In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [3]:
#path = 'M:/My Documents/Business/Bugisiw Ltd/TaxReturn/TaxReturn_2020-2021/LLP accounts/'
taxreturn_yr='TaxReturn_2022-2023'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'
startdatestr='2022-04-06'
enddatestr='2022-07-31'

In [4]:
startdate = pd.to_datetime(startdatestr).date()
enddate = pd.to_datetime(enddatestr).date()

In [5]:
input_file_6045 = path + '6045_categorisedAndBealsReconciled.xlsx'
df6=pd.read_excel(input_file_6045,sheet_name='6045')
df6=df6[~df6.Account.isnull()]
input_file_3072 = path + '3072_categorisedAndBealsReconciled.xlsx'
df3=pd.read_excel(input_file_3072,sheet_name='3072')
df3=df3[~df3.Account.isnull()]
#df3.loc[(df3.Description.isnull()),'Description']=''

In [6]:
input_file_beals = path + 'beals_bankReconciled.xlsx'
dfB=pd.read_excel(input_file_beals,sheet_name='beals')
dfB['ReconciledBankAccount']=dfB['ReconciledBankAccount'].astype('str')

### Transform Bank data

In [7]:
df6.rename(columns={'Account':'BankAccount'},inplace=True)
df3.rename(columns={'Account':'BankAccount'},inplace=True)
df6['PostNarrative'] = df6.Property.replace(np.nan,'') + ':6045:' + df6['Memo'].replace(np.nan,'') + '/' + df6['Description'].replace(np.nan,'')
df3['PostNarrative'] = df3.Property.replace(np.nan,'') + ':3072:' + df3['Memo'].replace(np.nan,'') + '/' + df3['Description'].replace(np.nan,'')
df6['Amount']=-df6['Amount']
df3['Amount']=-df3['Amount']

### Assign Accounting codes

In [8]:
config_file = path + 'accounts-config.xlsx'
dfCfg=pd.read_excel(config_file,sheet_name='config',converters={'MT6045':str,'IV3072':str})

In [9]:
dfCfg

,Id,Category,MT6045,IV3072,Natwest,AccountDescription
0,1,CouncilTax,3110,3110,NaN,Rates
1,2,Insurance,3170,3170,NaN,Premises Insurance
2,3,AgentFees,1570,1570,NaN,Management Fees
3,4,Maintenance,3140,3140,NaN,"Repairs, renewals, maintenance"
4,5,Mortgage,3631,3631,3631.0,Other interest payable
5,6,RentalIncome,0001,0002,NaN,Rental Income
6,7,ServiceCharge,3120,3120,NaN,Property management and service charges
7,8,Accounting,9999,9999,NaN,Accounting
8,10,Drawings,7501,7501,NaN,Drawings
9,11,Deposit,9999,9999,NaN,Deposit


In [10]:
cols=['BankAccount','Date','Property','PostNarrative','Amount','Account','Account Description','Category']
journals=path + 'journals/'

for idx, row in dfCfg.iterrows():
    df6.loc[(df6.Category==row.Category),'Account']=row.MT6045
    df6.loc[(df6.Category==row.Category),'Account Description']=row.AccountDescription
    df3.loc[(df3.Category==row.Category),'Account']=row.IV3072
    df3.loc[(df3.Category==row.Category),'Account Description']=row.AccountDescription
    
    if(df6[cols][(df6.Category==row.Category)].shape[0]>0):
        writer = pd.ExcelWriter(backup_file(journals + '/' + str(row.Id) + '.' + row.Category + '_6045.xlsx'), engine='xlsxwriter',datetime_format='dd/mm/yyyy')
        df6[cols][(df6.Category==row.Category)].to_excel(writer,index=False)
        writer.save()
    if(df3[cols][(df3.Category==row.Category)].shape[0]>0):
        writer = pd.ExcelWriter(backup_file(journals + '/' + str(row.Id) + '.' + row.Category + '_3072.xlsx'), engine='xlsxwriter',datetime_format='dd/mm/yyyy')
        df3[cols][(df3.Category==row.Category)].to_excel(writer,index=False)
        writer.save()